# Regression, Part B

### Baseline

In [1]:
# Import
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
import torch
import importlib_resources

from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from scipy import stats

from dtuimldmtools import draw_neural_net, train_neural_net, rlr_validate

In [2]:
# OFFLINE LOADING OF DATA
X = np.loadtxt('../wine/wine.data', delimiter=',')
y = X[:,0]
X = np.delete(X,0,axis=1)
# y = np.loadtxt('./wine/wine.names', delimiter=',')

Xorig = X
# Standardizing the data

X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# One of K encoding
# y = similarity.one_of_k(y)

# Ensure y values are within the range of the identity matrix's size
num_classes = int(np.max(y)) + 1
y2 = np.eye(num_classes)[y.astype(int)]

#removing first column of y
y2 = y2[:,1:]

#Appending y to X
X = np.append(X,y2,axis=1)

#Extracting the first column of X
y = X[:,0]



attributeNames = [
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

#Removing the first column of X
X = np.delete(X,0,axis=1)
# Removing alcohol from attributenames
attributeNames = attributeNames[1:]


N, M = X.shape

# Add offset attribute
X = np.concatenate((np.ones((X.shape[0], 1)), X), 1)
attributeNames = ["Offset"] + attributeNames + ["Class 1", "Class 2", "Class 3"]
M = M + 1


In [3]:
# Step 2: Set up K-Fold cross-validation (using outer loop)
K = 10
CV = model_selection.KFold(n_splits=K, shuffle=True)

# Initialize arrays to store errors for each fold
baseline_errors_train = np.empty(K)
baseline_errors_test = np.empty(K)

k = 0
for train_index, test_index in CV.split(X):
    print(f"Computing fold {k + 1}/{K}...")

    # Extract training and test sets for the current fold
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]

    # Step 3: Baseline model - predict the mean of y_train
    mean_y_train = np.mean(y_train)

    # Predict the mean for both training and test sets
    y_pred_train = np.full(len(y_train), mean_y_train)
    y_pred_test = np.full(len(y_test), mean_y_train)

    # Step 4: Calculate Mean Squared Error (MSE) for training and test sets
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Store the errors
    baseline_errors_train[k] = mse_train
    baseline_errors_test[k] = mse_test

    print(f"     Fold {k + 1}/{K} - MSE test: {mse_test:.4f}")

    k += 1

# Step 5: Print the results
print("\nBaseline Model Results:")
print(f"Average training MSE: {np.mean(baseline_errors_train):.4f}")
print(f"Average test MSE: {np.mean(baseline_errors_test):.4f}")

Computing fold 1/10...
     Fold 1/10 - MSE test: 1.1352
Computing fold 2/10...
     Fold 2/10 - MSE test: 1.1084
Computing fold 3/10...
     Fold 3/10 - MSE test: 0.6024
Computing fold 4/10...
     Fold 4/10 - MSE test: 1.1291
Computing fold 5/10...
     Fold 5/10 - MSE test: 0.8579
Computing fold 6/10...
     Fold 6/10 - MSE test: 1.4315
Computing fold 7/10...
     Fold 7/10 - MSE test: 1.2407
Computing fold 8/10...
     Fold 8/10 - MSE test: 0.8923
Computing fold 9/10...
     Fold 9/10 - MSE test: 0.7644
Computing fold 10/10...
     Fold 10/10 - MSE test: 0.8997

Baseline Model Results:
Average training MSE: 0.9996
Average test MSE: 1.0062


### Fit ANN model - Do not use, friends

In [4]:
# ANN
# Using script from: ex8_3_1 with modifications

# Define K-Fold cross-validation
K = 10
CV = model_selection.KFold(n_splits=K, shuffle=True)

# Define the model structure for regression
n_hidden_units = 5                            # Adjust as needed
model = lambda: torch.nn.Sequential(
    torch.nn.Linear(X.shape[1], n_hidden_units),    # Input layer to hidden layer
    torch.nn.ReLU(),                                # Activation function
    torch.nn.Linear(n_hidden_units, 1)              # Output layer with one output (regression)
)

# Use MSE loss function for regression
loss_fn = torch.nn.MSELoss()

# Initialize variables to store errors
Error_train = np.empty(K)
Error_test = np.empty(K)

k = 0
for train_index, test_index in CV.split(X):
    print(f"Computing CV fold {k + 1}/{K}...")

    # Extract training and test sets
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]

    # Convert data to torch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float).view(-1, 1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float).view(-1, 1)

    # Train the neural network
    net, _, _ = train_neural_net(
        model,
        loss_fn,
        X=X_train_tensor,
        y=y_train_tensor,
        n_replicates=3,
        max_iter = 1000
    )

    # Make predictions on the training and test sets
    y_train_pred = net(X_train_tensor).detach().numpy().flatten()
    y_test_pred = net(X_test_tensor).detach().numpy().flatten()

    # Calculate Mean Squared Error (MSE)
    Error_train[k] = mean_squared_error(y_train, y_train_pred)
    Error_test[k] = mean_squared_error(y_test, y_test_pred)

    print(f"     Fold {k + 1}/{K} - MSE test: {Error_test[k]:.4f}")

    k += 1

# Print the results
print("\nANN Model Results (Regression):")
print(f"Average training MSE: {np.mean(Error_train):.4f}")
print(f"Average test MSE: {np.mean(Error_test):.4f}")


Computing CV fold 1/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.31674185	0.00012662944
		Final loss:
		1000	0.31674185	0.00012662944

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.24613397	0.0003204611
		Final loss:
		1000	0.24613397	0.0003204611

	Replicate: 3/3
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.29696822	0.000315618
		Final loss:
		1000	0.29696822	0.000315618
     Fold 1/10 - MSE test: 0.4216
Computing CV fold 2/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.22796433	0.00039446008
		Final loss:
		1000	0.22796433	0.00039446008

	Replicate: 2/3
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.23435465	0.00022020581
		Final loss:
		1000	0.23435465	0.00022020581

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.26538295	0.00022948712
		Final loss:
		1000	0.26538295	0.00022948712
     Fold 2/10 - MSE test: 0.5688
Computing CV fold 3/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.28784746	0.00024179957
		Final loss:
		1000	0.28784746	0.00024179957

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.23437896	0.0005356688
		Final loss:
		1000	0.23437896	0.0005356688

	Replicate: 3/3
		Iter	Loss			Rel. loss


...

		1000	0.22278455	0.00024327198
		Final loss:
		1000	0.22278455	0.00024327198

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.21696997	0.0004702948
		Final loss:
		1000	0.21696997	0.0004702948
     Fold 8/10 - MSE test: 0.5374
Computing CV fold 9/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.25309876	0.00024968493
		Final loss:
		1000	0.25309876	0.00024968493

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.25241092	0.00019513271
		Final loss:
		1000	0.25241092	0.00019513271

	Replicate: 3/3
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.2766381	0.00025353298
		Final loss:
		1000	0.2766381	0.00025353298
     Fold 9/10 - MSE test: 0.7563
Computing CV fold 10/10...

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.26443398	0.0004850608
		Final loss:
		1000	0.26443398	0.0004850608

	Replicate: 2/3
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.27130598	0.00013794961
		Final loss:
		1000	0.27130598	0.00013794961

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.2894213	0.00031129076
		Final loss:
		1000	0.2894213	0.00031129076
     Fold 10/10 - MSE test: 0.2774

ANN Model Results (Regression):
Average training MSE: 0.2402
Average test MSE: 0.4985
